# Using Deep Learning Models in Gravitational Wave Physics

Start with some imports

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Empty, Queue

import numpy as np
import torch

import utils

In [2]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super().__init__()

        self.layers = torch.nn.ModuleList()
        for size in hidden_sizes:
            self.layers.append(torch.nn.Linear(input_size, size))
            self.layers.append(torch.nn.ReLU())
            input_size = size

        self.layers.append(torch.nn.Linear(input_size, 1))
        self.layers.append(torch.nn.Sigmoid())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

Use the model for inference in the normal way

In [3]:
model = MLP(64, [256, 128, 64])

x = np.random.randn(64).astype("float32")
with torch.no_grad():
    y = model(torch.tensor(x))
y.numpy()

array([0.50555277], dtype=float32)

Save and load the model to use at our leisure

In [4]:
torch.save(model.state_dict(), "model.pt")

model = MLP(64, [256, 128, 64])
with torch.no_grad():
    y_new = model(torch.tensor(x))
assert y_new != y

model.load_state_dict(torch.load("model.pt"))
with torch.no_grad():
    y_new = model(torch.tensor(x))
assert y_new == y

Inference on a chunk of data in batches

In [5]:
@torch.no_grad()
def do_some_inference(model, X, batch_size=8):
    dataset = torch.utils.data.TensorDataset(torch.tensor(X))
    for [x] in torch.utils.data.DataLoader(dataset, batch_size=batch_size):
        y = model(x)
        yield y.numpy()

In [6]:
X = np.random.randn(100000, 64).astype("float32")
with utils.get_progbar() as progbar:
    task_id = progbar.add_task("[cyan]Inference", total=len(X))
    outputs = []
    for y in do_some_inference(model, X):
        outputs.append(y)
        progbar.update(task_id, advance=len(y))

    output = np.concatenate(outputs, axis=0)
    progbar.console.log(output.shape)

Output()

[07:55:31] (100000, 1)                                  ]8;id=68539;file://C:\Users\amacg\AppData\Local\Temp\ipykernel_34348\2525482730.py\Temp/ipykernel_34348/2525482730.py]8;;\:10

In [7]:
def parallel_inference_task(q, X):
    model = MLP(64, [256, 128, 64])
    model.eval()
    for y in do_some_inference(model, X):
        q.put(y)

In [8]:
def parallel_inference(X, num_jobs, progbar):
    task_id = progbar.add_task(
        f"[cyan]Inference with {num_jobs} jobs", total=len(X)
    )

    q = Queue()
    with ThreadPoolExecutor(max_workers=num_jobs) as pool:
        for job, x in enumerate(np.array_split(X, num_jobs)):
            pool.submit(parallel_inference_task, q, x)

        outputs = []
        while len(outputs) < len(X):
            try:
                y = q.get_nowait()
                outputs.extend(y)
                progbar.update(task_id, advance=len(y))
            except Empty:
                time.sleep(0.01)
        return np.stack(outputs, axis=0)

In [9]:
with utils.get_progbar() as progbar:
    y = parallel_inference(X, 1, progbar)
    y = parallel_inference(X, 2, progbar)
    y = parallel_inference(X, 4, progbar)

Output()